In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

In [3]:
experiment_id = "8U2ir8mQR3yNMdX1QUMOcw"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
experiment

In [4]:
df = experiment.get_scalars()

KeyboardInterrupt: 

In [ ]:
print(df["run"].unique())
print(df["tag"].unique())

In [ ]:
dfw = df.pivot_table(
        values="value",
        index=["run", "step"],
        columns="tag",
        dropna=False,
    )
dfw = dfw.reset_index()
dfw.columns.name = None
dfw.columns.names = [None for name in dfw.columns.names]

In [ ]:
print(dfw["run"].unique())

In [ ]:
dfw["step"] = dfw["step"] * dfw["epoch"].max() / dfw["step"].max()

In [ ]:
df_scale_1 = dfw[dfw["run"].str.endswith("00.0-1.0-small/version_0") | dfw["run"].str.endswith("00.0-1.0-small/version_1") | dfw["run"].str.endswith("00.0-1.0-small/version_2")]

In [ ]:
print(df_scale_1["run"].unique())

In [ ]:
method_split = df_scale_1.run.apply(lambda run: f"{run.split('-')[1]}".upper())
method_split.name = None

In [ ]:
plt.figure(figsize=(4.8, 4.8))
sns.set(font_scale=1.5, style="white")
sns.lineplot(data=df_scale_1, x="step", y="valid_acc", style=method_split, color='black').set(title="Validation accuracy", ylabel="Accuracy (%)", xlabel="Trained epochs")
plt.tight_layout()
plt.savefig('nmnist_accuracy.svg')

In [ ]:
df_multi_scale = dfw[dfw["run"].str.endswith("00.0-1.0-small/version_0") | dfw["run"].str.endswith("00.0-0.1-small/version_0") | dfw["run"].str.endswith("00.0-0.01-small/version_0")]
print(df_multi_scale["run"].unique())

In [ ]:
exodus_df_1 = df_multi_scale[df_multi_scale["run"] == "nmnist-exodus-100000.0-1.0-small/version_0"]
exodus_df_01 = df_multi_scale[df_multi_scale["run"] == "nmnist-exodus-100000.0-0.1-small/version_0"]
exodus_df_001 = df_multi_scale[df_multi_scale["run"] == "nmnist-exodus-100000.0-0.01-small/version_0"]
slayer_df_1 = df_multi_scale[df_multi_scale["run"] == "nmnist-slayer-100000.0-1.0-small/version_0"]
slayer_df_01 = df_multi_scale[df_multi_scale["run"] == "nmnist-slayer-100000.0-0.1-small/version_0"]
slayer_df_001 = df_multi_scale[df_multi_scale["run"] == "nmnist-slayer-100000.0-0.01-small/version_0"]

gdf = pd.DataFrame({
    'algorithm': ['EXODUS', 'EXODUS', 'EXODUS', 'SLAYER', 'SLAYER', 'SLAYER', 'EXODUS', 'EXODUS', 'EXODUS', 'SLAYER', 'SLAYER', 'SLAYER', 'EXODUS', 'EXODUS', 'EXODUS', 'SLAYER', 'SLAYER', 'SLAYER'],
    'Layer': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3],
    'Gradient scaling': [1, 1, 1, 1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
    'Mean 2-norm': [
        exodus_df_1["grad_2.0_norm/network.2.weight_epoch"].mean(),
        exodus_df_1["grad_2.0_norm/network.4.weight_epoch"].mean(),
        exodus_df_1["grad_2.0_norm/network.6.weight_epoch"].mean(),
        slayer_df_1["grad_2.0_norm/fc1.weight_epoch"].mean(),
        slayer_df_1["grad_2.0_norm/fc2.weight_epoch"].mean(),
        slayer_df_1["grad_2.0_norm/fc3.weight_epoch"].mean(),
        exodus_df_01["grad_2.0_norm/network.2.weight_epoch"].mean(),
        exodus_df_01["grad_2.0_norm/network.4.weight_epoch"].mean(),
        exodus_df_01["grad_2.0_norm/network.6.weight_epoch"].mean(),
        slayer_df_01["grad_2.0_norm/fc1.weight_epoch"].mean(),
        slayer_df_01["grad_2.0_norm/fc2.weight_epoch"].mean(),
        slayer_df_01["grad_2.0_norm/fc3.weight_epoch"].mean(),
        exodus_df_001["grad_2.0_norm/network.2.weight_epoch"].mean(),
        exodus_df_001["grad_2.0_norm/network.4.weight_epoch"].mean(),
        exodus_df_001["grad_2.0_norm/network.6.weight_epoch"].mean(),
        slayer_df_001["grad_2.0_norm/fc1.weight_epoch"].mean(),
        slayer_df_001["grad_2.0_norm/fc1.weight_epoch"].mean(),
        slayer_df_001["grad_2.0_norm/fc3.weight_epoch"].mean(),
        ]
})
gdf

In [ ]:
sns.set(font_scale=0.9, style="white")
plt.figure(figsize=(4, 8))
g = sns.FacetGrid(gdf, col="algorithm", hue="Gradient scaling", palette='flare')
g.map(plt.plot, "Layer", "Mean 2-norm", linestyle="-", marker="o")
g.set_titles("{col_name}")
plt.suptitle("Gradient norms")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
plt.yscale("log")
# for ax in g.axes.flatten():
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0, title="Surrogate scale")
plt.tight_layout()
# plt.subplots_adjust(top=0.8)
plt.savefig(f"nmnist_mean_gradnorm.svg")

NameError: name 'sns' is not defined